This function computes the interpolant for the Lagrangian linear momentum barrier equation:

 \begin{equation}
 \mathbf{x}_0'=\nu \rho \mathbf{J} \mathbf{\nabla_0 \overline{\omega (\mathbf{F_{t_0}^{t}(x_0)}, t)}},
 \label{eq: linearmomentum} 
 \end{equation}
 
 where $ \nu $ is the viscosity, $ \rho $ the density of the fluid, $ \mathbf{J} = \begin{pmatrix} 0 && 1 \\ -1 && 0 \end{pmatrix} $ and $ \mathbf{\omega} $ is the vorticity. $ \overline{(\ldots)} $ denotes a temporal average.
 
 For simplicity we can normalize equation $ \ref{eq: linearmomentum} $ by $ \nu $ and $ \rho $ and thus simply omit them for simplicity as they will play no role when exploring the structures of the field $ \mathbf{x'} $.

| Name | Type (Shape) | Description |
| --- | --- | --- |
| X_domain | array (Ny, Nx)| X-meshgrid of initial conditions |
| Y_domain | array (Ny, Nx)| Y-meshgrid with Ny=Nx=1024 over which $ \omega(\mathbf{x}, t) $ is defined |
| X | array (1024, 1024)| X-meshgrid over which $ \omega(\mathbf{x}, t) $ is defined |
| Y | array (1024, 1024)| Y-meshgrid over which $ \omega(\mathbf{x}, t) $ is defined |
| omega | array (1024, 1025, 251) | $ \omega(\mathbf{x}, t) $ |
| Fmap | array (Ny, Nx, N) | $ \mathbf{F}_{t_0}^{t}(\mathbf{x}_0) $, with $ t \in [t_0, t_1] $  |
| time | array (N, ) | $ t \in [t_0, t_1] $ |
| dt_data| float| resolution of time-inteval of data |
| aux_grid | list (2,) | aux_grid[0]: dx_auxiliary spacing <br /> aux_grid[1]: dy_auxiliary spacing |
| Interpolant | list (2,) | Interpolant[0]: Interpolant for x-component of $ \mathbf{x}' $ <br /> Interpolant[1]: Interpolant for y-component of $ \mathbf{x}' $ |

In [1]:
# import Rectangular bivariate spline from scipy
from scipy.interpolate import RectBivariateSpline as RBS

# import package for progress bar
from tqdm.notebook import tqdm

# Import numpy 
import numpy as np

In [2]:
def LagrangianActiveMomentum(X_domain, Y_domain, X, Y, omega, Fmap, time, dt_data, aux_grid):
    
    # auxiliary grid
    rho_x = aux_grid[0]
    rho_y = aux_grid[1]
    
    # iterate over all trajectories and compute omega along them
    
    # compute interpolant for omega over meshgrid X, Y over time
    Interpolant_omega = []
    for i in tqdm(range(omega.shape[2])):
        Interpolant_omega.append(RBS(Y[:,0], X[0,:], omega[:,:,i]))
    
    Omega = np.zeros((Fmap.shape[0], Fmap.shape[2]))
    counter = 0
    
    # evaluate omega at 'x' at time 't' 
    # iterate over time:
    for i in time:
        
        k = int(i/dt_data)
        
        if k == Fmap.shape[0]-1:
            
            Omega[counter,:] = Interpolant_omega[k](Fmap[counter,1,:].ravel(), Fmap[counter,0,:].ravel(), grid = False)
        
        else:
            
            Omegai = Interpolant_omega[k](Fmap[counter,1,:].ravel(), Fmap[counter,0,:].ravel(), grid = False)
            Omegaf = Interpolant_omega[k+1](Fmap[counter,1,:].ravel(), Fmap[counter,0,:].ravel(), grid = False)
            Omega[counter,:] = ((k+1)*dt_data-i)/dt_data*Omegai + (i-k*dt_data)/dt_data*Omegaf
        
        counter += 1

    Omega_avg = np.mean(Omega, axis = 0)
    Omega_avg = Omega_avg.reshape((X_domain.shape[0], Y_domain.shape[1]))
    
    # Interpolant for average of omega
    Interpolant_omega_avg = RBS(Y_domain[:,0], X_domain[0,:], Omega_avg)
    
    Jgrad_omega_avg = np.zeros((X_domain.shape[0], Y_domain.shape[1], 2))*np.nan
        
    # compute x'=J grad(average(omega)), with J = [[0, 1], [-1, 0]]
    for i in range(X_domain.shape[0]):
        
        for j in range(Y_domain.shape[1]):
            
            # apply periodic boundary conditions to point 'x'
            x = np.array([X_domain[i, j], Y_domain[i, j]])
            
            # evaluate grad(omega) at 'x' using auxiliary grid
            xR = (x[0] + rho_x)%(X[0,-1]-X[0,0])
            OmegaR = Interpolant_omega_avg(x[1], xR)[0][0]
            
            xL = (x[0] - rho_x)%(X[0,-1]-X[0,0])
            OmegaL = Interpolant_omega_avg(x[1], xL)[0][0]
            
            xU = (x[1] + rho_y)%(Y[-1,0]-Y[0,0])
            OmegaU = Interpolant_omega_avg(xU, x[0])[0][0]
            
            xD = (x[1] - rho_y)%(Y[-1,0]-Y[0,0])
            OmegaD = Interpolant_omega_avg(xD, x[0])[0][0]
            
            Jgrad_omega_avg[i, j, 0] = -(OmegaU - OmegaD)/(2*rho_y)
            Jgrad_omega_avg[i, j, 1] = (OmegaR - OmegaL)/(2*rho_x)
    
    Interpolant = []
    Interpolant.append(RBS(Y_domain[:,0], X_domain[0,:], Jgrad_omega_avg[:,:,0]))
    Interpolant.append(RBS(Y_domain[:,0], X_domain[0,:], Jgrad_omega_avg[:,:,1]))
    
    return Interpolant